In [ ]:
# coding:utf-8
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def gragh(result, epoch):    
    x = np.arange(0, epoch, 1)
    left = np.array(x)
    height = np.array(result)
    plt.plot(left, height)

In [ ]:
from deep_qnetwork_with_cnn import DeepQNetworkWithCNN
import random
import time
import copy
import numpy as np

ACTION = [0, 1, 2, 3]
NUM_HID1 = 3136
NUM_HID2 = 512
NUM_OUT  = 4
BATCH_SIZE = 32
EPOCH = 10000
GAMMA = 0.9
EPSIL = 1.0
MINEPSIL = 0.1
NUM_LIMIT_RECORD = 10**4
NUM_INIT_RECORD = 10**5
ProccerUnit = "CPU"
MODEL_NAME = "./DQN_model/breakout1.model"
RESULT_REWARD = []

STATE_LENGTH = 4  # 状態を構成するフレーム数
FRAME_WIDTH = 84  # リサイズ後のフレーム幅
FRAME_HEIGHT = 84  # リサイズ後のフレーム高さ
NO_OP_STEPS = 30

def agent_action(env, action):
    next_state, reward, terminal, info = env.step(action)
    return next_state, reward, terminal, info

def deep_q_learn(env, agent):
    agent.load_weight()
    agent.update_target_network()
    step = 0
    for episode in range(1, EPOCH):
        terminal = False
        sum_reward = 0
        next_obs = env.reset()  # 環境の初期化、初期画面を返す
        for i in range(random.randint(1, NO_OP_STEPS)):  # ランダムなフレーム数分「何もしない」行動で遷移させる
            now_obs = next_obs
            next_obs, _, _, _ = env.step((i%4))  # 「何もしない」行動を取り、次の画面を返す
        now_state = agent.get_initial_state(now_obs, next_obs)  # 初期状態を作る
        last = 5
        learn_flg = 0
        epsil = agent.EPSIL
        if episode % 4 == 0:
            agent.EPSIL = 0.05
            learn_flg = 1
        while(not terminal):
            # 状態更新
            now_obs = copy.deepcopy(next_obs)
            # 方策に従って行動を選択
            action, _, action_list = agent.policy_egreedy(now_state, agent.Q_network, step)
            # 行動による次状態を観測
            next_obs, reward, term, info = agent_action(env, ACTION[action])
            # ゴールの設定
            terminal, reward = (True, reward) if term else (False, reward)
            
            if last != info["ale.lives"]:
                reward = -1.0
            
            preprocess_obs = agent.pre_proccesing(now_obs, next_obs)
            next_state = np.append(now_state[1:, :, :], np.array([preprocess_obs]), axis=0)
            sum_reward += reward
            agent.stock_record(now_state, action, next_state, reward, terminal, action_list)

            env.render()
            if terminal:
                break
                
            # 入力状態更新
            now_state = copy.deepcopy(next_state)
            last = info["ale.lives"]
            
            # Experience_Replayによるバッチ学習
            if len(agent.record) > NUM_LIMIT_RECORD:
                agent.init_record()
                if learn_flg:
                    agent.experience_replay_ver2()
            if agent.EPSIL > MINEPSIL: agent.EPSIL -= EPSIL / EPOCH / 10
            if step % 10**4 == 0:
                agent.update_target_network()               
            step += 1
            if step % 500 == 0: print("step:" + str(step))
        
        print(episode, sum_reward, agent.EPSIL, step)
        agent.EPSIL = epsil
        RESULT_REWARD.append(sum_reward)
        agent.save_weight()
    gragh(RESULT_REWARD, len(RESULT_REWARD))

if __name__=="__main__":
    import gym
    env = gym.make("Breakout-v0")
    agent = DeepQNetworkWithCNN(NUM_HID1, NUM_HID2, NUM_OUT,ProccerUnit, EPSIL, GAMMA, BATCH_SIZE, MODEL_NAME)
    deep_q_learn(env, agent)

[2017-06-11 22:41:18,601] Making new env: Breakout-v0


[0, 1, 2, 3]
1 -3.0 0.9973200000000122 268
2 -4.0 0.9979000000000096 478
step:500
3 -4.0 0.9976800000000106 710
step:1000
step:1500
4 -5.0 0.05 1588
5 -2.0 0.9972100000000127 1867
step:2000
6 -4.0 0.9977500000000102 2092
7 -4.0 0.99780000000001 2312
step:2500
8 -4.0 0.05 2656
9 -4.0 0.9979800000000092 2858
step:3000
10 -3.0 0.9970900000000132 3149
11 -4.0 0.998230000000008 3326
step:3500
12 -4.0 0.05 3702
13 -3.0 0.9972500000000125 3977
step:4000
14 -5.0 0.9985100000000068 4126
step:4500
15 0.0 0.9954000000000209 4586
step:5000
16 -3.0 0.05 5190
17 -5.0 0.9984100000000072 5349
step:5500
18 -4.0 0.9977200000000104 5577
19 -5.0 0.9984400000000071 5733
step:6000
20 -4.0 0.05 6128
21 -4.0 0.9980500000000089 6323
step:6500
22 -4.0 0.9980400000000089 6519
23 -4.0 0.9975600000000111 6763
step:7000
step:7500
24 -2.0 0.05 7505
25 -5.0 0.9985600000000066 7649
26 -5.0 0.9983500000000075 7814
step:8000
27 -4.0 0.998020000000009 8012
step:8500
28 -4.0 0.05 8730
29 -5.0 0.9983700000000074 8893
step: